# Boosting

다양한 머신러닝 기법 중에 특히 좋은 성능을 보이는 기법으로 앙상블이 있다. 앙상블은 단일 모델이 아닌 여러 모델을 활용하여 더 나은 결과를 도출하는 기법이다.

앙상블 기법은 크게 Bagging 과 Boosting 이 있는데, 본문에서는 Boosting 에 대해 알아볼 것이다.

Bagging 에서 여러 모델의 투표를 통해 결과를 예측한다면, Boosting 은 하나의 모델 $A$가 예측했을 때의 오차를 다시 다른 모델 $B$를 예측 하고, 이를 반복 하는 기법이다.

이 때 $A$ 와 $B$ 를 weak learner 라고 한다. 즉 weak learner 여러개를 더하여 strong learner 를 만드는 것이 Boosting 이라 할 수 있다.

## Boosting 의 종류

대표적인 Boosting 알고리즘으로 Adaboost, Gradient Boosting, XGBoost, LightGBM 등이 있다. Adaboost 가 1997년에 개발된 최초의 부스팅 알고리즘이다. Adaboost 는 이전 weak learner 가 틀렸던 샘플에 대해 높은 가중치를 주어 오차를 줄이는 기법이고, Gradient Boost 는 이전 weak learner 의 오차를 데이터로 삼아 새로운 weak learner 에 적합 시킨 후 합하는 방식이다.

부스팅에서는 일반적으로 의사결정나무(DT)를 weak learner 로 사용한다. 따라서 먼저 DT를 구현 하고 실제 데이터로 확인 해보자.

주의해야 할 점은 DT를 구현할 때 Boosting 에 적용 시킬 수 있게 하기 위하여 가중치가 적용 될 수 있게끔 구현한다.

In [447]:
import numpy as np
import pandas as pd
import pickle
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from pprint import pprint

In [448]:
class DT:
    def __init__(self, max_depth=2, weights=None):
        self.max_depth = max_depth
        self.weights = weights
        
    def fit(self, X, y):
        self.target_size = np.max(y) + 1
        self.tree = self._make_tree(X, y, self.max_depth, self.weights)
    
    def _gini(self, y, weights):
        unique, cnt = np.unique(y, return_counts=True)
        if weights is not None:
            cnt = np.array([np.sum(weights[np.where(y==i)[0]]) for i in unique])
        cnt = cnt / cnt.sum()
        return 1 - np.sum(cnt * cnt)
    
    def _make_tree(self, X, y, depth, weights):
        # leaf node
        if depth == 0 or len(np.unique(y)) == 1:
            prob = np.zeros(self.target_size)
            unique, cnt = np.unique(y, return_counts=True)
            if self.weights is not None:
                cnt = np.array([weights[np.where(y==i)[0]].sum() for i in unique])
            prob[unique] = cnt / cnt.sum()
            return { "leaf": True, "prob": prob, "size": len(y) }

        min_gini = 1
        min_col = 0
        split = None

        for i, col in enumerate(X.T):
            u = np.sort(np.unique(col))
            u = (u[1:] + u[:-1]) / 2

            for val in u:
                lt, rt = np.where(col <= val), np.where(col > val)
                y_lt, y_rt = y[lt], y[rt]
                if weights is None:
                    cur_gini = (len(y_lt)*self._gini(y_lt, None) + len(y_rt)*self._gini(y_rt, None)) / (len(y))
                else:
                    w_lt, w_rt = weights[lt], weights[rt]
                    cur_gini = (w_lt.sum()*self._gini(y_lt, w_lt) + w_rt.sum()*self._gini(y_rt, w_rt)) / weights.sum()
                    
                if cur_gini < min_gini:
                    min_gini, min_col, split = cur_gini, i, val

        lt = np.where(X[:, min_col] <= split)[0]
        rt = np.where(X[:, min_col] > split)[0]
        return {
            "split": (min_col, split),
            "lt": self._make_tree(X[lt, :], y[lt], depth-1, weights[lt] if weights is not None else None),
            "rt": self._make_tree(X[rt, :], y[rt], depth-1, weights[rt] if weights is not None else None),
            "leaf": False,
            "size": len(y)
        }
    
    def _predict_one(self, x):
        cur = self.tree
        while cur["leaf"] == False:
            col, split = cur["split"]
            cur = cur["lt"] if x[col] <= split else cur["rt"]
        return cur["prob"]
    
    def predict(self, X):
        return np.apply_along_axis(self._predict_one, 1, X)

In [449]:
loaded = load_iris()
data, target = loaded["data"], loaded["target"]
X_tr, X_tst, y_tr, y_tst = train_test_split(data, target, test_size=0.4)

X_tr.shape

(90, 4)

In [450]:
tree = DT(max_depth=3)
tree.fit(X_tr, y_tr)
pprint(tree.tree)

{'leaf': False,
 'lt': {'leaf': True, 'prob': array([1., 0., 0.]), 'size': 34},
 'rt': {'leaf': False,
        'lt': {'leaf': False,
               'lt': {'leaf': True, 'prob': array([0., 1., 0.]), 'size': 25},
               'rt': {'leaf': True,
                      'prob': array([0.        , 0.33333333, 0.66666667]),
                      'size': 3},
               'size': 28,
               'split': (3, 1.65)},
        'rt': {'leaf': False,
               'lt': {'leaf': True,
                      'prob': array([0.        , 0.33333333, 0.66666667]),
                      'size': 3},
               'rt': {'leaf': True, 'prob': array([0., 0., 1.]), 'size': 25},
               'size': 28,
               'split': (3, 1.7000000000000002)},
        'size': 56,
        'split': (2, 4.85)},
 'size': 90,
 'split': (2, 2.45)}


In [451]:
y_prob = tree.predict(X_tst)
y_pred = np.argmax(y_prob, 1)
(y_pred == y_tst).sum() / len(y_tst)

0.95

In [452]:
loaded = load_breast_cancer()
data, target = loaded["data"], loaded["target"]
X_tr, X_tst, y_tr, y_tst = train_test_split(data, target, test_size=0.4)

tree = DT(max_depth=3)
tree.fit(X_tr, y_tr)

y_prob = tree.predict(X_tst)
y_pred = np.argmax(y_prob, 1)
(y_pred == y_tst).sum() / len(y_tst)

0.9385964912280702

본문에서는 Adaboost or Adaptive Boosting 을 구현한다. Adaboost 는 다음과 같이 작동한다.

다음 시행을 반복한다.
1. weak learner 로 depth 가 1인 DT, 즉 stump 를 사용한다. 이 모델을 $M_{i-1}$ 이라 하자. 초기 샘플 가중치 $w_0$는 모든 샘플에 대해 같다. 즉 $w_0=1/n$. 따라서 $i=0$일 경우 $M_0$을 원래 데이터 $\{(x,y)\}_0$에 적합시킨다.
2. Amount of Say 를 구한다. $$ AOS = \frac{1}{2}\log(\frac{1-Total Error}{Total Error}) $$
3. 샘플 가중치를 다음과 같이 업데이트 한다.
    1. 모델이 정답을 맞춘 샘플일 경우 $w_i = w_{i-1} * e^{-AOS}$
    2. 그렇지 않을 경우 $w_i = w_{i-1} * e^{AOS}$
4. 새로운 샘플 가중치에 따라 샘플링 하여 새로운 데이터 $\{(x,y)\}_i$ 를 만든다.

위 시행을 반복하면 여러개의 weak learner 와 그에 해당하는 Amount Of Say 가 나온다. 예측 할 때 각 Amount Of Say 를 가중치로 두어 확률을 구하고 가장 큰 항목을 정답으로 선택한다.

In [453]:
class AdaBoost:
    def __init__(self, n_estimators=10):
        self.learners = []
        self.n_estimators = n_estimators
    
    def _aos(self, stump):
        err = 0
        if stump["leaf"] == False:
            lt, rt = stump["lt"], stump["rt"]
            err = 1 - (np.max(lt["prob"]) * lt["size"] + np.max(rt["prob"]) * rt["size"]) / stump["size"]
        return np.log((1-err+1e-15)/(err+1e-15))/2
        
    def fit(self, X, y):
        w = np.ones_like(y) / len(y)
        self.learners.append(DT(max_depth=1, weights=w))
        self.aos = []
        data = X
        label = y
        
        for learner in self.learners:
            # Step 1
            learner.fit(data, label)
            
            # Step 2
            aos = self._aos(learner.tree)
            self.aos.append(aos)
            if len(self.aos) == self.n_estimators:
                break
                
            # Step 3
            prob = learner.predict(data)
            w[np.where(np.argmax(prob, axis=1) == label)] *= np.exp(-aos)
            w[np.where(np.argmax(prob, axis=1) != label)] *= np.exp(aos)
            w = w / np.sum(w)
            
            # Step 4 (DT 자체에 샘플 가중치를 적용 시키는 것이 더 좋으므로 생략)
            #idx = np.random.choice(len(label), len(label), replace=True, p=w)
            #data = X[idx, :]
            #label = y[idx]
            self.learners.append(DT(max_depth=1, weights=w))
        self.aos = np.array(self.aos)
            
    def _predict_one(self, x):
        probs = np.array([learner._predict_one(x) for learner in self.learners])
        w = self.aos / np.sum(self.aos)
        return w.dot(probs)
        
    def predict(self, X):
        return np.apply_along_axis(self._predict_one, 1, X)

In [454]:
loaded = load_iris()
data, target = loaded["data"], loaded["target"]
X_tr, X_tst, y_tr, y_tst = train_test_split(data, target, test_size=0.4)

X_tr.shape

(90, 4)

In [455]:
boost = AdaBoost()
boost.fit(X_tr, y_tr)
boost.aos

array([0.39746494, 0.4252533 , 0.47291081, 0.38910173, 0.69000059,
       0.46212709, 0.58721052, 0.64257156, 0.50355015, 0.4692775 ])

In [456]:
y_prob = boost.predict(X_tst)
y_pred = np.argmax(y_prob, 1)
(y_pred == y_tst).sum() / len(y_tst)

0.9666666666666667

In [457]:
loaded = load_breast_cancer()
data, target = loaded["data"], loaded["target"]
X_tr, X_tst, y_tr, y_tst = train_test_split(data, target, test_size=0.4)

boost = AdaBoost()
boost.fit(X_tr, y_tr)
boost.aos

y_prob = boost.predict(X_tst)
y_pred = np.argmax(y_prob, 1)
(y_pred == y_tst).sum() / len(y_tst)

0.9473684210526315

타이타닉 데이터에 대해서 DT 와 AdaBoost 를 적용시켜 다음을 비교해보자.
1. 직접 구현한 DT
2. sklearn의 DT
2. 직접 구현한 AdaBoost
3. sklearn의 AdaBoost

In [458]:
# 편의상 이미 전처리 되어있는 titanic 데이터를 이용한다.
tr = pd.read_csv("data/titanic_tr.csv")
tst = pd.read_csv("data/titanic_tst.csv")
tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
0,0,2,0,-0.592148,1,0,-0.502163,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1,0,1,0.638430,1,0,0.786404,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1,2,1,-0.284503,0,0,-0.488580,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1,0,1,0.407697,1,0,0.420494,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,2,0,0.407697,0,0,-0.486064,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [459]:
data = np.array(tr.drop(["Survived"], axis=1))
target = np.array(tr["Survived"])

data.shape

(891, 15)

In [460]:
X_tr, X_tst, y_tr, y_tst = train_test_split(data, target, test_size=0.3)

X_tr.shape

(623, 15)

In [461]:
# 직접 구현 한 DT
tree = DT(max_depth=3)
tree.fit(X_tr, y_tr)

y_prob = tree.predict(X_tst)
y_pred = np.argmax(y_prob, axis=1)
(y_pred == y_tst).sum() / len(y_tst)

0.8134328358208955

In [462]:
# sklearn 의 DT
treemodel = DecisionTreeClassifier()
treemodel.fit(X_tr, y_tr)

y_pred = treemodel.predict(X_tst)
(y_pred == y_tst).sum() / len(y_tst)

0.7985074626865671

In [463]:
# 직접 구현한 AdaBoost
boost = AdaBoost()
boost.fit(X_tr, y_tr)

y_prob = boost.predict(X_tst)
y_pred = np.argmax(y_prob, axis=1)
(y_pred == y_tst).sum() / len(y_tst)

0.7649253731343284

In [464]:
# sklearn 의 AdaBoost
boostmodel = AdaBoostClassifier()
boostmodel.fit(X_tr, y_tr)

y_pred = boostmodel.predict(X_tst)
(y_pred == y_tst).sum() / len(y_tst)

0.8134328358208955

Boosting 은 weak learner 가 풀지 못하는 문제, 즉 어려운 문제에 집중하기 때문에 데이터가 복잡할 수록 더 좋은 성능을 낸다. 하지만 위 데이터는 복잡하지 않은 데이터라서 DT 와 큰 성능차이를 보이지 않는다.

sklearn의 모델은 여러 최적화가 들어가 있어 속도와 예측력 측면에서 직접 구현한 코드보다 더 좋은 성능을 보인다.